## Update OiDB metadata using JMMC's CatalogAPI

In [1]:
import pyvo
from a2p2.jmmc import Catalog

prod=False
prod=True

if True:
    oidbMainUrl="http://oidb.jmmc.fr"
    oidbTapUrl="http://tap.jmmc.fr/vollt/tap"
else:    
    oidbMainUrl="http://oidb-beta.jmmc.fr"
    oidbTapUrl="http://tap-preprod.jmmc.fr/vollt/tap"

oidbTap = pyvo.dal.TAPService(oidbTapUrl)
oidbCatalog = Catalog("oidb", prod=prod)

INFO - a2p2.jmmc.catalogs  - 2022-03-25 14:30:33,582 - catalogs.py:44 - Create catalog wrapper to access 'oidb' (production API at https://oidb.jmmc.fr/restxq/catalogs)


### Define helper function to fix content

In [2]:
collectionToDataPi={
    "371c145f-7889-47d5-9c14-5f7889c7d510":"KRAUS",
    "c5a5d133-a178-4949-a5d1-33a1781949e5":"SOULAIN"
}

def fixDataPi(granule):
    """Use a hardcoded mapping to fix datapi automatically if the value in database have to be updated.
    returns: none if notheing has to be changed else a dict. that can be given to catalog.updateRows().
    """
    collection_id=granule['obs_collection']
    if collection_id in collectionToDataPi:                
        dataPI_g = granule['datapi']
        dataPI_c = collectionToDataPi[collection_id]
        if dataPI_g != dataPI_c:
            granule_id = granule['id']            
            return {"id": str(granule_id), "datapi": dataPI_c} # force str to avoid TypeError: Object of type int64 is not JSON serializable
    else:
        print(f"Can't fix dataPI, please complete collectionToDataPi dict. with \"{collection_id}\":\"PINAME\"")
        print(f"  visit {oidbMainUrl}/collection.html?id={collection_id}")
        
def fixRecords(query):
    """ Walk accross granules selected by given query and return a list of dict so we can update the oidb catalog using api.updateRows()"""
    updates=[]
    granules = oidbTap.search(query).to_table()
    for granule in granules:
        j = fixDataPi(granule)
        if j:
            updates+=(j)
    return updates

    #oidbCatalog

### Fix some known cases

In [3]:
updates=[] # accumulates json update orders

# Fix case 1 : Avoid bad datapis
dataPiToFix="Guillaume Mella" # this guy can't be a datapi, he often does hesitate between RA and DEC
query = f"SELECT ALL * FROM oidb AS t WHERE ( t.datapi LIKE '%{dataPiToFix}%' )"
case1updates = fixRecords(query)

        
# Fix case 2 : Update SUV collections so we leave JMMC colleague as collection_creator but fix dataPi since we can't yet deposit data for another people.
suv_collections=["c5a5d133-a178-4949-a5d1-33a1781949e5"] # we may get full list using a query...
case2updates=[]
for collection_id in suv_collections:
    query = f"SELECT ALL * FROM oidb AS t WHERE obs_collection='{collection_id}'"
    case2updates+=fixRecords(query)
    
# Apply updates if we have orders    
for updates in (case1updates,case2updates):
    if len(updates)>0:
        print(f"Submitting {len(updates)} through catalogAPI")
        oidbCatalog.updateRows(updates)

# That's all folks !!!